# Génération du Dataset Synthétique
## Prédiction des moments de vie - Banque française

Ce notebook génère un dataset synthétique de 10,000 clients avec :
- Données démographiques et financières
- Comportements bancaires
- 8 moments de vie à prédire
- Signaux comportementaux faibles

## 1. Imports et Configuration

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Fixer la seed pour la reproductibilité
np.random.seed(42)
random.seed(42)

print("Imports réussis")

Imports réussis


## 2. Paramètres du Dataset

In [2]:
# Paramètres principaux
N_CLIENTS = 10000
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2024, 12, 31)

# Définition des moments de vie à prédire
MOMENTS_DE_VIE = {
    "mariage": {"prob": 0.03, "age_range": (25, 40)},
    "naissance": {"prob": 0.04, "age_range": (25, 45)},
    "achat_immobilier": {"prob": 0.05, "age_range": (25, 55)},
    "changement_emploi": {"prob": 0.08, "age_range": (22, 60)},
    "retraite": {"prob": 0.02, "age_range": (60, 70)},
    "deces_proche": {"prob": 0.03, "age_range": (30, 80)},
    "divorce": {"prob": 0.02, "age_range": (30, 60)},
    "creation_entreprise": {"prob": 0.01, "age_range": (25, 55)},
}

print(f"Configuration:")
print(f"   - Nombre de clients: {N_CLIENTS:,}")
print(f"   - Période: {START_DATE.date()} à {END_DATE.date()}")
print(f"   - Moments de vie: {len(MOMENTS_DE_VIE)}")

Configuration:
   - Nombre de clients: 10,000
   - Période: 2023-01-01 à 2024-12-31
   - Moments de vie: 8


## 3. Fonctions de Génération

In [3]:
def generate_clients():
    """Génère les données clients de base"""

    clients = []

    for client_id in range(1, N_CLIENTS + 1):
        age = np.random.normal(45, 15)
        age = max(18, min(85, age))  # Limiter entre 18 et 85 ans

        # Génération des caractéristiques démographiques
        client = {
            "client_id": f"CLI_{client_id:06d}",
            "age": int(age),
            "genre": np.random.choice(["H", "F"], p=[0.48, 0.52]),
            "situation_familiale": np.random.choice(
                ["celibataire", "marie", "divorce", "veuf"], p=[0.35, 0.45, 0.15, 0.05]
            ),
            "nb_enfants": np.random.choice(
                [0, 1, 2, 3, 4], p=[0.35, 0.25, 0.25, 0.10, 0.05]
            ),
            "csp": np.random.choice(
                [
                    "cadre",
                    "employe",
                    "ouvrier",
                    "profession_liberale",
                    "retraite",
                    "etudiant",
                ],
                p=[0.20, 0.35, 0.15, 0.10, 0.15, 0.05],
            ),
            "region": np.random.choice(
                ["IDF", "PACA", "ARA", "Occitanie", "HDF", "Autre"],
                p=[0.25, 0.12, 0.12, 0.10, 0.10, 0.31],
            ),
            "anciennete_banque_mois": int(np.random.exponential(60) + 6),
        }

        # Génération des caractéristiques financières
        if client["csp"] == "cadre":
            revenu_base = np.random.normal(4500, 1500)
        elif client["csp"] == "profession_liberale":
            revenu_base = np.random.normal(5500, 2000)
        elif client["csp"] == "employe":
            revenu_base = np.random.normal(2500, 800)
        elif client["csp"] == "ouvrier":
            revenu_base = np.random.normal(2000, 600)
        elif client["csp"] == "retraite":
            revenu_base = np.random.normal(1800, 500)
        else:  # etudiant
            revenu_base = np.random.normal(800, 300)

        client["revenu_mensuel"] = max(0, revenu_base)
        client["epargne_totale"] = max(
            0, np.random.exponential(client["revenu_mensuel"] * 12)
        )
        client["credits_en_cours"] = np.random.choice(
            [0, 1, 2, 3], p=[0.50, 0.30, 0.15, 0.05]
        )

        # Génération des comportements bancaires
        client["nb_produits_bancaires"] = np.random.choice(
            [1, 2, 3, 4, 5], p=[0.15, 0.30, 0.30, 0.20, 0.05]
        )
        client["utilise_app_mobile"] = np.random.choice([0, 1], p=[0.30, 0.70])
        client["nb_connexions_mois"] = int(
            np.random.exponential(15)
            if client["utilise_app_mobile"]
            else np.random.exponential(3)
        )

        # Moyennes transactionnelles (derniers 6 mois)
        client["montant_moyen_transactions"] = max(0, np.random.normal(1500, 800))
        client["nb_transactions_mois"] = int(np.random.normal(25, 15))
        client["ratio_depenses_revenus"] = min(
            1.5, max(0.3, np.random.normal(0.75, 0.15))
        )

        # Contacts avec la banque
        client["nb_appels_conseiller_6mois"] = np.random.choice(
            [0, 1, 2, 3, 4, 5], p=[0.40, 0.30, 0.15, 0.08, 0.05, 0.02]
        )
        client["nb_visites_agence_6mois"] = np.random.choice(
            [0, 1, 2, 3], p=[0.50, 0.30, 0.15, 0.05]
        )

        clients.append(client)

    return pd.DataFrame(clients)


print("Fonction generate_clients() définie")

Fonction generate_clients() définie


In [4]:
def generate_life_events(clients_df):
    """Génère les moments de vie pour chaque client"""

    life_events = []

    for _, client in clients_df.iterrows():
        client_age = client["age"]

        # Pour chaque moment de vie, déterminer si le client le vit
        for event_type, event_params in MOMENTS_DE_VIE.items():
            age_min, age_max = event_params["age_range"]
            base_prob = event_params["prob"]

            # Ajuster la probabilité selon l'âge
            if age_min <= client_age <= age_max:
                prob = base_prob
            else:
                prob = (
                    base_prob * 0.1
                )  # Probabilité réduite hors de la tranche d'âge optimale

            # Déterminer si l'événement se produit
            if np.random.random() < prob:
                # Date de l'événement (dans les 12 prochains mois)
                days_offset = np.random.randint(0, 365)
                event_date = START_DATE + timedelta(days=days_offset)

                life_events.append(
                    {
                        "client_id": client["client_id"],
                        "moment_de_vie": event_type,
                        "date_evenement": event_date,
                        "horizon_prediction": "12_mois",
                    }
                )

    return pd.DataFrame(life_events)


print("Fonction generate_life_events() définie")

Fonction generate_life_events() définie


In [12]:
def add_behavioral_signals(clients_df, life_events_df):
    """Ajoute des signaux comportementaux réalistes et bruités"""
    
    # Initialiser toutes les colonnes de signaux à 0
    signal_columns = [
        "recherche_pret_perso_recent",
        "augmentation_epargne_recente",
        "ouverture_compte_epargne_recent",
        "consultation_assurance_vie",
        "simulation_pret_immobilier",
        "variation_revenus_recente",
        "consultation_placements",
        "consultation_pret_pro",
    ]
    
    for col in signal_columns:
        clients_df[col] = 0
    
    # 1. Ajouter des signaux PROBABILISTES pour les clients avec événements
    # Ces signaux ont une PROBABILITÉ d'apparaître, pas une certitude
    for _, event in life_events_df.iterrows():
        client_id = event["client_id"]
        event_type = event["moment_de_vie"]
        idx = clients_df[clients_df["client_id"] == client_id].index[0]

        # Signaux avec probabilité réduite (50-70% au lieu de 100%)
        if event_type == "mariage":
            if np.random.random() < 0.60:  # 60% de chance seulement
                clients_df.loc[idx, "recherche_pret_perso_recent"] = 1
            if np.random.random() < 0.50:
                clients_df.loc[idx, "augmentation_epargne_recente"] = 1

        elif event_type == "naissance":
            if np.random.random() < 0.65:
                clients_df.loc[idx, "ouverture_compte_epargne_recent"] = 1
            if np.random.random() < 0.55:
                clients_df.loc[idx, "consultation_assurance_vie"] = 1

        elif event_type == "achat_immobilier":
            if np.random.random() < 0.70:
                clients_df.loc[idx, "simulation_pret_immobilier"] = 1
            if np.random.random() < 0.60:
                clients_df.loc[idx, "augmentation_epargne_recente"] = 1

        elif event_type == "changement_emploi":
            if np.random.random() < 0.50:
                clients_df.loc[idx, "variation_revenus_recente"] = 1

        elif event_type == "retraite":
            if np.random.random() < 0.65:
                clients_df.loc[idx, "consultation_placements"] = 1

        elif event_type == "creation_entreprise":
            if np.random.random() < 0.70:
                clients_df.loc[idx, "consultation_pret_pro"] = 1
    
    # 2. Ajouter du BRUIT : des signaux pour des clients SANS événement
    # Cela simule des faux positifs (gens qui consultent sans aller au bout)
    clients_with_events = life_events_df["client_id"].unique()
    clients_without_events = clients_df[~clients_df["client_id"].isin(clients_with_events)]
    
    # Pour chaque client sans événement, petite probabilité d'avoir des signaux
    for idx in clients_without_events.index:
        # 5-10% de chance d'avoir chaque signal (bruit de fond)
        if np.random.random() < 0.08:
            clients_df.loc[idx, "simulation_pret_immobilier"] = 1
        if np.random.random() < 0.07:
            clients_df.loc[idx, "augmentation_epargne_recente"] = 1
        if np.random.random() < 0.06:
            clients_df.loc[idx, "consultation_placements"] = 1
        if np.random.random() < 0.05:
            clients_df.loc[idx, "recherche_pret_perso_recent"] = 1
        if np.random.random() < 0.05:
            clients_df.loc[idx, "consultation_assurance_vie"] = 1
        if np.random.random() < 0.04:
            clients_df.loc[idx, "consultation_pret_pro"] = 1
        if np.random.random() < 0.06:
            clients_df.loc[idx, "variation_revenus_recente"] = 1
        if np.random.random() < 0.05:
            clients_df.loc[idx, "ouverture_compte_epargne_recent"] = 1

    return clients_df


print("Fonction add_behavioral_signals() définie")

Fonction add_behavioral_signals() définie


## 4. Génération du Dataset

In [13]:
print("Génération du dataset bancaire - Prédiction des moments de vie")
print("=" * 70)

# Générer les clients
print("\nGénération des données clients...")
clients_df = generate_clients()
print(f"   {len(clients_df):,} clients générés")

# Aperçu des données
print("\nAperçu des premières lignes:")
display(clients_df.head())

Génération du dataset bancaire - Prédiction des moments de vie

Génération des données clients...
   10,000 clients générés

Aperçu des premières lignes:
   10,000 clients générés

Aperçu des premières lignes:


,client_id,age,genre,situation_familiale,nb_enfants,csp,region,anciennete_banque_mois,revenu_mensuel,epargne_totale,credits_en_cours,nb_produits_bancaires,utilise_app_mobile,nb_connexions_mois,montant_moyen_transactions,nb_transactions_mois,ratio_depenses_revenus,nb_appels_conseiller_6mois,nb_visites_agence_6mois
0,CLI_000001,51,H,marie,0,ouvrier,IDF,46,2340.875490,36185.721244,1,5,1,4,1613.945103,3,0.746767,0,1
1,CLI_000002,30,H,celibataire,1,employe,IDF,76,3488.229435,27063.294343,0,1,0,0,792.945538,43,0.675874,0,0
2,CLI_000003,37,H,divorce,1,etudiant,Autre,56,831.133826,22003.780519,0,3,0,3,2092.745793,36,0.587196,0,2
3,CLI_000004,46,F,marie,2,cadre,IDF,310,3990.968188,86641.514165,0,2,1,9,3741.542797,47,0.579222,0,1
4,CLI_000005,46,F,marie,0,cadre,IDF,207,3431.919432,130216.176674,1,3,1,8,480.431486,-5,0.665629,2,1


In [14]:
# Informations sur le dataset
print("Informations sur le dataset:")
print(f"   - Shape: {clients_df.shape}")
print(f"   - Colonnes: {list(clients_df.columns)}")
print(f"\nStatistiques descriptives:")
display(clients_df.describe())

Informations sur le dataset:
   - Shape: (10000, 19)
   - Colonnes: ['client_id', 'age', 'genre', 'situation_familiale', 'nb_enfants', 'csp', 'region', 'anciennete_banque_mois', 'revenu_mensuel', 'epargne_totale', 'credits_en_cours', 'nb_produits_bancaires', 'utilise_app_mobile', 'nb_connexions_mois', 'montant_moyen_transactions', 'nb_transactions_mois', 'ratio_depenses_revenus', 'nb_appels_conseiller_6mois', 'nb_visites_agence_6mois']

Statistiques descriptives:


,age,nb_enfants,anciennete_banque_mois,revenu_mensuel,epargne_totale,credits_en_cours,nb_produits_bancaires,utilise_app_mobile,nb_connexions_mois,montant_moyen_transactions,nb_transactions_mois,ratio_depenses_revenus,nb_appels_conseiller_6mois,nb_visites_agence_6mois
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,44.743400,1.246500,65.172100,2936.318512,35139.581562,0.746600,2.708800,0.693600,10.714800,1506.809951,24.385500,0.749651,1.13820,0.750500
std,14.405031,1.163646,59.128656,1719.253231,46134.298677,0.886604,1.098874,0.461021,13.427028,773.041660,14.952139,0.151025,1.28991,0.884154
min,18.000000,0.000000,6.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-34.000000,0.300000,0.00000,0.000000
25%,34.000000,0.000000,23.000000,1778.918614,7529.625473,0.000000,2.000000,0.000000,2.000000,960.059442,14.000000,0.648278,0.00000,0.000000
50%,44.000000,1.000000,47.000000,2475.443892,19874.309269,0.000000,3.000000,1.000000,6.000000,1500.548663,24.000000,0.750543,1.00000,1.000000
75%,55.000000,2.000000,89.000000,3692.781246,44381.145140,1.000000,4.000000,1.000000,15.000000,2032.182767,35.000000,0.852956,2.00000,1.000000
max,85.000000,4.000000,527.000000,11686.882683,807997.809165,3.000000,5.000000,1.000000,127.000000,5120.438640,77.000000,1.300209,5.00000,3.000000


In [15]:
# Générer les moments de vie
print("Génération des moments de vie...")
life_events_df = generate_life_events(clients_df)
print(f"   {len(life_events_df):,} événements générés")

# Distribution des événements
print("\nDistribution des moments de vie:")
event_distribution = life_events_df["moment_de_vie"].value_counts()
for event, count in event_distribution.items():
    pct = count / len(life_events_df) * 100
    print(f"   - {event}: {count} ({pct:.1f}%)")

print("\nAperçu des événements:")
display(life_events_df.head(10))

Génération des moments de vie...
   1,745 événements générés

Distribution des moments de vie:
   - changement_emploi: 618 (35.4%)
   - achat_immobilier: 348 (19.9%)
   - deces_proche: 250 (14.3%)
   - naissance: 192 (11.0%)
   - divorce: 128 (7.3%)
   - mariage: 99 (5.7%)
   - creation_entreprise: 83 (4.8%)
   - retraite: 27 (1.5%)

Aperçu des événements:


,client_id,moment_de_vie,date_evenement,horizon_prediction
0,CLI_000005,achat_immobilier,2023-01-17,12_mois
1,CLI_000013,deces_proche,2023-12-08,12_mois
2,CLI_000014,mariage,2023-02-08,12_mois
3,CLI_000024,achat_immobilier,2023-06-13,12_mois
4,CLI_000024,changement_emploi,2023-06-30,12_mois
5,CLI_000025,changement_emploi,2023-08-03,12_mois
6,CLI_000043,mariage,2023-04-24,12_mois
7,CLI_000045,naissance,2023-02-24,12_mois
8,CLI_000052,changement_emploi,2023-09-25,12_mois
9,CLI_000053,deces_proche,2023-09-06,12_mois


In [16]:
# Ajouter les signaux comportementaux
print("Ajout des signaux comportementaux...")
clients_df = add_behavioral_signals(clients_df, life_events_df)
print(f"   Signaux ajoutés")

print("\nDataset final avec signaux:")
display(clients_df.head())

Ajout des signaux comportementaux...
   Signaux ajoutés

Dataset final avec signaux:
   Signaux ajoutés

Dataset final avec signaux:


,client_id,age,genre,situation_familiale,nb_enfants,csp,region,anciennete_banque_mois,revenu_mensuel,epargne_totale,...,nb_appels_conseiller_6mois,nb_visites_agence_6mois,recherche_pret_perso_recent,augmentation_epargne_recente,ouverture_compte_epargne_recent,consultation_assurance_vie,simulation_pret_immobilier,variation_revenus_recente,consultation_placements,consultation_pret_pro
0,CLI_000001,51,H,marie,0,ouvrier,IDF,46,2340.875490,36185.721244,...,0,1,0,0,0,0,0,0,0,0
1,CLI_000002,30,H,celibataire,1,employe,IDF,76,3488.229435,27063.294343,...,0,0,0,0,0,0,0,0,0,0
2,CLI_000003,37,H,divorce,1,etudiant,Autre,56,831.133826,22003.780519,...,0,2,0,0,0,0,0,0,0,0
3,CLI_000004,46,F,marie,2,cadre,IDF,310,3990.968188,86641.514165,...,0,1,0,0,0,0,0,0,0,0
4,CLI_000005,46,F,marie,0,cadre,IDF,207,3431.919432,130216.176674,...,2,1,0,1,0,0,0,0,0,0


## 5. Sauvegarde des Données

In [17]:
# Sauvegarder les datasets
print("Sauvegarde des datasets...")
clients_df.to_csv("../data/clients_data.csv", index=False)
life_events_df.to_csv("../data/life_events.csv", index=False)
print("   clients_data.csv")
print("   life_events.csv")

Sauvegarde des datasets...
   clients_data.csv
   life_events.csv


## 6. Statistiques Finales

In [18]:
print("Statistiques du dataset:")
print(f"   - Nombre de clients: {len(clients_df):,}")
print(f"   - Nombre de features: {len(clients_df.columns)}")
print(f"   - Nombre d'événements: {len(life_events_df):,}")
print(
    f"   - Clients avec au moins 1 événement: {life_events_df['client_id'].nunique():,}"
)
print(
    f"   - Taux de clients avec événement: {life_events_df['client_id'].nunique() / len(clients_df) * 100:.1f}%"
)

print("\nDataset généré avec succès!")

Statistiques du dataset:
   - Nombre de clients: 10,000
   - Nombre de features: 27
   - Nombre d'événements: 1,745
   - Clients avec au moins 1 événement: 1,603
   - Taux de clients avec événement: 16.0%

Dataset généré avec succès!
